In [1]:
from __future__ import division
from __future__ import print_function
from __future__ import unicode_literals

In [2]:
import uscensus
from uscensus.util.sqlalchemycache import SqlAlchemyCache

In [3]:
from configparser import RawConfigParser
import qgrid
qgrid.nbinstall()

In [4]:
cp = RawConfigParser()

In [5]:
cp.read('C:/cygwin64/home/nkrishna/.census')

['C:/cygwin64/home/nkrishna/.census']

In [ ]:
# Work around for some Windows TLS issues

import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.poolmanager import PoolManager
import ssl

class MyAdapter(HTTPAdapter):
    def init_poolmanager(self, connections, maxsize, block=False):
        self.poolmanager = PoolManager(num_pools=connections,
                                       maxsize=maxsize,
                                       block=block,
                                       ssl_version=ssl.PROTOCOL_TLSv1_2)
s = requests.Session()
s.mount('https://', MyAdapter())

In [ ]:
%%time
with SqlAlchemyCache('sqlite://') as cache:
    cl = uscensus.DiscoveryInterface(
        cp.get('census','api_key'),
        cache,
        session=s,
        index=True,
        vintage='2015/acs'
    )

In [ ]:
cl.search('ACS')

In [ ]:
results = _
acs1 = results[1]

In [ ]:
acs1.searchVariables('"median gross rent"')

In [ ]:
acs1.concepts

In [ ]:
api=cl.search('title:ACS 1-Year Supplemental Estimates')[0]

In [ ]:
api.searchVariables('median rent')

In [ ]:
api.variables['K202511_001E']

In [ ]:
med_rent_by_county = api(['K202511_001E'], geo_for={'county': '*'})
med_rent_by_county.set_index(['state', 'county'], inplace=True)
med_rent_by_county.sort_index(inplace=True)
med_rent_by_county

In [ ]:
api.vintage

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import fiona
import geopandas as gpd
import shapely

In [ ]:
%%time
# Generated in GetCountyShapes.ipynb
counties_df = gpd.GeoDataFrame.from_file('counties.geojson',
                                         driver='GeoJSON')
counties_df.crs = fiona.crs.from_epsg(4326)

In [ ]:
continental = shapely.geometry.box(-125.0011, 24.9493, -66.9326, 49.5904)
counties_df[counties_df.geometry.within(continental)].simplify(0.1).plot(figsize=((15,15)))

In [ ]:
%time
counties_df.set_index(['STATE', 'COUNTY'], inplace=True)
counties_df.sort_index(inplace=True)

In [ ]:
%time
counties_df.index.names=['state','county']
joined = counties_df.join(med_rent_by_county)

In [ ]:
#joined.total_bounds
continental = shapely.geometry.box(-125.0011, 24.9493, -66.9326, 49.5904)
joined_cont = joined[joined.intersects(continental)]

In [ ]:
axes=joined_cont.plot(column='K202511_001E', figsize=(15,15))
axes.figure.dpi=190
axes

In [ ]:
joined_cont.head()

In [ ]:
import matplotlib as mpl
import numpy as np
inc_min=joined_cont['K202511_001E'].min()
inc_max=joined_cont['K202511_001E'].max()
X = [[0, 0], [.9, .9]]
plt.gca().imshow(X, interpolation='bicubic', cmap=mpl.cm.get_cmap(),
          extent=(0, .1, 0, 1), alpha=1)